In [ ]:
cmap = ['magenta','blue', 'green', 'red', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan']

In [ ]:
import pprint as pp
import numpy as np
import time
import os
import sys
import random


import matplotlib.pyplot as plt
%matplotlib inline 
import matplotlib
from matplotlib.patches import Patch

import shutil
from numpy import genfromtxt
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable

torch.set_default_dtype(torch.float64)

import itertools
import numpy.linalg  as lin

import cProfile, pstats

from collections import OrderedDict
torch.set_num_threads(1) #cpu num
from fractions import Fraction
import seaborn as sns
import pandas as pd
import math
from collections import defaultdict

In [ ]:
def avg_alpha(_aa):
    aa_ep = defaultdict(list)
    for i, x in enumerate(_aa):
        aa_ep[math.ceil(x[0])].append(i)
        
    _aa = np.array(_aa)
        
    _a = []
    _areal = []
    _amax = []
    
    for key in sorted(aa_ep.keys()):
        ind = aa_ep[key]
        _a.append(np.mean(_aa[ind,2]))
        _areal.append(np.mean(_aa[ind,3]))
        _amax.append(np.mean(_aa[ind,4]))
    return list(sorted(aa_ep.keys())),_a, _areal, _amax
    

def fillup(_epp,_ep,_l):
    
    for ind in range(1,len(_epp)):
        if _epp[ind] not in _ep:
            _l.insert(ind,_l[ind-1])
    return _l    

def runmax(_l):
    
    return [max(_l[:i+1]) for i in range(len(_l))]


In [ ]:
%cd ../Logs

In [ ]:
DNAME = ['ijcnn1','rcv1','real-sim','news20','covtype']
TICK=[5,10,5,10,5]
ALGO = ['ai_sarah']

In [ ]:
logfolder = '%s/reg/%s/'

In [ ]:
adf={}
for dname in DNAME:
    aDF = []
    print(dname)
    
    temp_logfolder = logfolder%(dname,'ai_sarah')
    for f in os.listdir(temp_logfolder):
        if 'DONE' in f or '.tar' not in f or 'RUN' in f:
            continue
        else:
            temp = torch.load(temp_logfolder+f)
            parm = temp['parm']
            seed = parm[1]
            bs = parm[0]
            alpha = temp['alpha']
            aEP,aL,arL,amL = avg_alpha(alpha)
            
            aDF+=[['AI-SARAH', gamma, bs, seed, e, 'alpha', ai] for e,ai\
                in zip(aEP,aL)]
            aDF+=[['AI-SARAH', gamma, bs, seed, e, 'alpha_newton', ali] for e,ali\
                in zip(aEP,arL)]
            aDF+=[['AI-SARAH', gamma, bs, seed, e, 'alpha_max', ami] for e,ami\
                in zip(aEP,amL)]
            
    adf[dname] = pd.DataFrame(data=aDF,columns=['algo','gamma','BS','seed',\
                                       'ep','type','alpha'])
    adf[dname] = adf[dname].sort_values(by=['algo','type'])

In [ ]:
%cd ../Plot

In [ ]:
sns.set()
sns.set(context="paper",font='serif')
sns.set_style("ticks")
fig, ax = plt.subplots(1,len(DNAME),figsize=(2.5*len(DNAME),4),sharex=False,sharey=False)
ind = 0

palette = sns.color_palette()[:2]
palette = cmap[:2]
markers = ('o', 'v')
dashes = ("","")
dashes = [(2,2)]*2

for d in range(len(DNAME)):
    
    if d==0:
        LEGEND='full'
    else:
        LEGEND=False
    
    subdf = adf[DNAME[d]][(adf[DNAME[d]]['ep']<=ENDEP[d])&(adf[DNAME[d]]['ep']>=1)\
                          &(adf[DNAME[d]]['type']!='alpha_newton')]
    lp=sns.lineplot(x='ep',y='alpha',dashes=dashes,palette=palette,\
       markers=markers,zorder=-1,markevery=5,markersize=8,lw=2.0,\
       legend=LEGEND,hue='type',ci=95,style='type',\
       data=subdf,ax=ax[d])
    ax[d].set_xlim([0,ENDEP[d]+0.1])
    ax[d].set_ylabel('')
    ax[d].set_title(DNAME[d],fontsize=15)
    ax[d].set_xlabel('')
    ax[d].grid('both')
    label=[r'$\alpha$',r'$\alpha_{max}$']
ax[d].set_xlabel('Effective Pass',fontsize=12)
handles, labels = ax[0].get_legend_handles_labels()
ax[0].legend(handles=handles[1:], \
             labels=label,fancybox=True,fontsize=12,loc='lower right')
fig.tight_layout()
plt.show()    

In [ ]:
exit(0)